# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [28]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Загрузка и подготовка данных

На данном данные загружаются и разбиваются на обучающую и валидационную выборки, в соотношении 75:25

In [29]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [30]:
data_1.sample(5)

,id,f0,f1,f2,product
23830,LnihI,0.536930,-0.499839,1.585133,101.234625
17775,JSkzB,0.076925,-0.212972,6.594024,142.839474
42971,HAUog,0.377430,-0.282113,-1.134120,56.845192
99560,9RPh0,0.940057,0.748740,-1.276136,33.679779
18120,d6ZlH,-0.989872,0.530342,5.835365,147.447134


In [31]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [32]:
data_2.sample(5)

,id,f0,f1,f2,product
41059,MyQyz,-0.857045,-3.081804,3.999712,110.992147
43761,YcEZS,6.493443,-0.711439,3.002242,80.859783
27037,Oazt2,7.843738,-8.669030,4.994159,134.766305
49343,twQVL,6.349549,-2.201355,3.000749,80.859783
93751,i8N8U,-11.652990,-8.215475,0.997027,30.132364


In [33]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [34]:
data_3.sample(5)

,id,f0,f1,f2,product
88259,KIoiR,0.741738,0.436720,3.902632,40.495971
45631,lULyQ,1.951993,-1.523091,-0.144337,75.655319
29171,9J3Sz,-1.961073,1.142530,2.190700,116.396151
50125,NgEY1,1.030945,3.810242,3.054304,69.142771
44911,eLHbN,-2.680819,-0.139307,0.511025,30.011320


In [35]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [36]:
target_1 = data_1['product']
features_1 = data_1.drop(['id', 'product'] , axis=1)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345) 

In [37]:
target_2 = data_2['product']
features_2 = data_2.drop(['id', 'product'] , axis=1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345) 

In [38]:
target_3 = data_3['product']
features_3 = data_3.drop(['id', 'product'] , axis=1)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size=0.25, random_state=12345) 

**Вывод**
Данные подготовлены для обучения модели, разбиты на 2 выборки - тестовую и валидационную в отношении 75:25

## Обучение и проверка модели

На данном этапе происходит обучение и проверка моделей.По условию задачи, для обучения модели подходит только линейная регрессия. Необхадимо найти средний запас предсказанного сырья и RMSE модели.

In [39]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)
print('Средний запас предсказанного сырья в 1-м регионе:', np.mean(predictions_valid_1))
print('RMSE модели для 1-го региона:',mean_squared_error(target_valid_1, predictions_valid_1)**0.5) 

Средний запас предсказанного сырья в 1-м регионе: 92.59256778438038
RMSE модели для 1-го региона: 37.5794217150813


In [40]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)
print('Средний запас предсказанного сырья во 2-м регионе:', np.mean(predictions_valid_2))
print('RMSE модели для 2-го региона:',mean_squared_error(target_valid_2, predictions_valid_2)**0.5) 

Средний запас предсказанного сырья во 2-м регионе: 68.728546895446
RMSE модели для 2-го региона: 0.893099286775616


In [41]:
model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)
predictions_valid_3 = model_3.predict(features_valid_3)
print('Средний запас предсказанного сырья в 3-м регионе:', np.mean(predictions_valid_3))
print('RMSE модели для 3-го региона:',mean_squared_error(target_valid_3, predictions_valid_3)**0.5) 

Средний запас предсказанного сырья в 3-м регионе: 94.96504596800489
RMSE модели для 3-го региона: 40.02970873393434


**Вывод:**
Качество 1 и 3 модели низкое, однако среднее значениие высокие, в то время как качество 2-й модели хорошее, но среднее значение меньше. 

In [42]:
features_train = [features_train_1, features_train_2, features_train_3]
target_train = [target_train_1, target_train_2, target_train_3]
features_valid = [features_valid_1, features_valid_2, features_valid_3]
target_valid = [target_valid_1, target_valid_2, target_valid_3]
model = LinearRegression()
predictions_valid = []
for i in range(3):
    model.fit(features_train[i], target_train[i])
    predictions_valid_i = model.predict(features_valid[i])
    predictions_valid.append(predictions_valid_i)
    print(f'Средний запас предсказанного сырья в {i+1} регионе: {np.mean(predictions_valid)}')
    print(f'RMSE модели для {i+1}-го региона: {mean_squared_error(target_valid[i], predictions_valid_i)**0.5}')   

Средний запас предсказанного сырья в 1 регионе: 92.59256778438038
RMSE модели для 1-го региона: 37.5794217150813
Средний запас предсказанного сырья в 2 регионе: 80.6605573399132
RMSE модели для 2-го региона: 0.893099286775616
Средний запас предсказанного сырья в 3 регионе: 85.42872021594377
RMSE модели для 3-го региона: 40.02970873393434


## Подготовка к расчёту прибыли

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [43]:
BUDGET = 10000000000
REVENUE_PER_PRODUCT = 450000 
QUANTITY = 200

In [44]:
stock = BUDGET/(REVENUE_PER_PRODUCT*QUANTITY)

In [45]:
stock

111.11111111111111

**Вывод**
Для того чтобы окупить бюджет на разработку 200 скважин в регионе, достаточный объём сырья для безубыточной разработки новой скважины должен быть 111.1 тыс баррелей. Средний запас предсказанного сырья в 3-х регионах меньше необходимого:
- в 1-м регионе 92.6 
- во 2-м регионе 68.7
- в 3-м регионе 94.9

## Расчёт прибыли и рисков 

**Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели:**

In [46]:
def profit(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return REVENUE_PER_PRODUCT * selected.sum() - BUDGET

**Расчет рисков и прибыли для каждого региона:**

Посчитайте риски и прибыль для каждого региона:
- Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

In [47]:
target_subsample_1 = target_valid_1.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
probs_subsample_1 = pd.Series(predictions_valid_1[target_subsample_1.index], index = target_subsample_1.index)

print(target_subsample_1 )
print(probs_subsample_1 )

NameError: name 'state' is not defined

In [54]:
state = np.random.RandomState(12345)
    
values = []
for i in range(1000):
    # < напишите код здесь>
    target_subsample_1 = target_valid_1.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
    probs_subsample_1 = pd.Series(predictions_valid_1[target_subsample_1.index], index = target_subsample_1.index)
    values.append(profit(target_subsample_1, probs_subsample_1, QUANTITY))

values = pd.Series(values)
lower = values.quantile(0.025)   
upper = values.quantile(0.975)

mean = values.mean()
loss = (values < 0).mean()

print("Средняя прибыль в 1-м регионе:", mean)
print(f'95%-й доверительный интервал: от {lower} до {upper}')
print('Риск убытков: ', loss)

Средняя прибыль в 1-м регионе: 425938526.9105923
95%-й доверительный интервал: от -102090094.83793654 до 947976353.358369
Риск убытков:  0.06


In [55]:
state = np.random.RandomState(12345)
    
values = []
for i in range(1000):
    # < напишите код здесь>
    target_subsample_2 = target_valid_2.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
    probs_subsample_2 = pd.Series(predictions_valid_2[target_subsample_2.index], index = target_subsample_2.index)
    values.append(profit(target_subsample_2, probs_subsample_2, QUANTITY))
    
values = pd.Series(values)
lower = values.quantile(0.025)   
upper = values.quantile(0.975)

mean = values.mean()
loss = (values < 0).mean()

print("Средняя прибыль во 2-м регионе:", mean)
print(f'95%-й доверительный интервал: от {lower} до {upper}')
print('Риск убытков: ', loss)

Средняя прибыль во 2-м регионе: 515222773.44328994
95%-й доверительный интервал: от 68873225.37050177 до 931547591.2570496
Риск убытков:  0.01


In [56]:
state = np.random.RandomState(12345)
    
values = []
for i in range(1000):
    # < напишите код здесь>
    target_subsample_3 = target_valid_3.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
    probs_subsample_3 = pd.Series(predictions_valid_3[target_subsample_3.index], index = target_subsample_3.index)
    values.append(profit(target_subsample_3, probs_subsample_3, QUANTITY))
    
values = pd.Series(values)
lower = values.quantile(0.025)   
upper = values.quantile(0.975)

mean = values.mean()
loss = (values < 0).mean()

print("Средняя прибыль в 3-м регионе:", mean)
print(f'95%-й доверительный интервал: от {lower} до {upper}')
print('Риск убытков: ', loss)

Средняя прибыль в 3-м регионе: 435008362.78275555
95%-й доверительный интервал: от -128880547.32978901 до 969706954.1802679
Риск убытков:  0.064


**Вывод** Было произведено исследование, наилучшие результаты показал 2-й регион

In [59]:
def bootstrap_func (target_valid, predictions_valid,i):
    state = np.random.RandomState(12345)
    
    values = []
    for _ in range(1000):
        # < напишите код здесь>
        target_subsample = target_valid.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        probs_subsample = pd.Series(predictions_valid[target_subsample.index], index = target_subsample.index)
        values.append(profit(target_subsample, probs_subsample, QUANTITY))

    values = pd.Series(values)
    lower = values.quantile(0.025)   
    upper = values.quantile(0.975)

    mean = values.mean()
    loss = (values < 0).mean()

    print(f"Средняя прибыль в {i+1}-м регионе:", mean)
    print(f'95%-й доверительный интервал: от {lower} до {upper}')
    print('Риск убытков: ', loss)
    print()

In [60]:
for i in range(3):
    bootstrap_func(target_valid[i], predictions_valid[i],i)

Средняя прибыль в 1-м регионе: 425938526.9105923
95%-й доверительный интервал: от -102090094.83793654 до 947976353.358369
Риск убытков:  0.06

Средняя прибыль в 2-м регионе: 515222773.44328994
95%-й доверительный интервал: от 68873225.37050177 до 931547591.2570496
Риск убытков:  0.01

Средняя прибыль в 3-м регионе: 435008362.78275555
95%-й доверительный интервал: от -128880547.32978901 до 969706954.1802679
Риск убытков:  0.064



## Общий вывод по исследованию:

1. Данные по скважинам в 3-х регионах, содержащие по 10000 записей ыли подготовлены для обучения моделей: разделены на обучающую и валидационную выборки в соотношении 75:25.
2. Для 3-х регионов были построены модели машинного обучения (LinearRegression) и обучены на основании тестовых выборок. Получены предсказания запасов нефти на основании валидационной выборки.
3. Написана функция, вычисляющая среднюю прибыль, исходя из условия:
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
4.  Проведен анализ возможной прибыли и риски техникой Bootstrap, получены следующие данные:
- 1-й регион: средняя прибыль - 426 млн руб., 95% интервал от -102 млн руб. до 948 млн руб., вероятность убытков - 6% 
- 2-й регион: средняя прибыль - 515 млн руб., 95% интервал от 68,9 млн руб. до 931,5 млн руб., вероятность убытков - 1%
- 3-й регион: средняя прибыль - 435 млн руб., 95% интервал от -128,8 млн руб. до 969,7 млн руб., вероятность убытков - 6,4%

Исходя, из полученных результатов, только 2-й регион имеет вероятность убытков меньше заданного порога 2,5%, а также самую высокую среднюю прибыль. 

Рекомендация к освоению - 2-й регион.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован